Example of using [`vtreat`](https://github.com/WinVector/pyvtreat) inside a sklearn pipeline.

This note is to bring out that while `vtreat` can be placed in such pipelines, one should not place it in pipelines used for hyperparameters search.  Intead during hyperparameters search we advise treating `vtreat` as a separate pre-processing step. If one wishes to play with different variable filter parameters, we suggest allowing `vtreat` to land excess variables and then filtering at a later stage.

For our demonstration we first load packages/modules.

In [1]:
import pandas
import numpy
import numpy.random
import vtreat
import vtreat.util
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

We set our pseudorandom state to improve reproducibilty.

In [2]:
numpy.random.seed(2019)

We new build our example training data.  It is designed to be a data set with categorical variables where common levels are informative and rare levels are not.  So setting at what rarity levels are encoded can be useful.

In [3]:
def make_data(nrows, 
              *,
              ncols=10,
              n_common_levels=5,
              n_rare_levels=10,
              rare_ratio=0.3,
              noise_magnitude=3.3,
              na_rate=0.1):
    # build a system of more common levels, which have signal,
    # and rare levels, which do not have signal
    common_levels = ['c_' + str(i) for i in range(n_common_levels)]
    rare_levels = ['r_' + str(i) for i in range(n_rare_levels)]
    levels = common_levels + rare_levels
    probs = numpy.asarray([1.0 / len(common_levels)] * len(common_levels) + 
                          [rare_ratio / len(rare_levels)] * len(rare_levels))
    probs = probs / sum(probs)
    effects = numpy.random.choice(
        [-1, 1], 
        size = len(common_levels), 
        replace=True).tolist() + [0]*len(rare_levels)
    effects = {li: ei for (li, ei) in zip(levels, effects)}
    # use this to populate up a data frame
    d = pandas.DataFrame({
        'x_' + str(i): numpy.random.choice(levels, 
                                           size=nrows, 
                                           replace=True, 
                                           p=probs) for i in range(ncols)
    })
    # build y
    y = noise_magnitude * numpy.random.normal(size=nrows)
    for i in range(ncols):
        y = y + d[d.columns[i]].map(effects)
    # introduce some NaNs
    if na_rate > 0:
        for i in range(ncols):
            idx = numpy.where(
                numpy.random.choice([False, True], 
                                    size=nrows, 
                                    replace=True, 
                                    p=[1 - na_rate, na_rate]))[0]
            if len(idx) > 0:
                d.loc[idx, d.columns[i]] = numpy.nan
    return d, y > 0

d_x, d_y = make_data(2000)

In [4]:
d_x.head()

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9
0,c_3,c_1,NaN,r_8,r_3,c_0,c_2,r_3,c_4,c_1
1,c_3,NaN,r_9,c_3,c_3,c_1,c_2,c_3,c_0,c_3
2,NaN,c_3,c_4,c_0,c_1,r_0,c_1,r_9,c_3,c_0
3,c_4,c_2,c_3,r_9,c_0,r_0,r_7,c_1,c_2,r_1
4,c_2,c_1,r_1,NaN,c_1,c_3,c_4,c_3,c_0,c_0


In [5]:
d_y.head()

0    False
1     True
2     True
3     True
4    False
dtype: bool

`vtreat` can be placed in a `scikit` `Pipeline` to be part of a reusable processing workflow.  However, it is *not* recommended to use such a pipeline for hyper-parameter search, as by design `vtreat` has very few tunable parameters (mostly just `indicator_min_fraction`), `vtreat` has its own out of sample simulation, and it is quite wasteful to re-compute similar data re-encodings again and again. Our advice is: treat `vtreat` as an early non-tuned pre-processing step and re-use its work in later pipelines 

To even place `vtreat` into hyperparameter search we need an adapter class that hides the non-tunable parameters. `vtreat`'s bundling of parameters into a re-usable object isn't compatible with the clone-step many hyper parameter searches use. `vtreat` does implement `get_parameters()` and `set_parameters()`, but for neatness it doesn't expose each individual possible setting a explicit constructor arugments (prefering a params object).

`vtreat` does try and support hyper-parameter tuning: for example it defaults to re-using cross validation plans where possible to de-noise the type of repeated application we may see in hyper-parameter optimization.

The additional adaption is easy and can be performed as follows.  Remember this adaption is not recommended. Instructions how to use `vtreat` can be found [here](https://github.com/WinVector/pyvtreat/blob/main/Examples/Classification/Classification.md).

In [6]:
class BinomialOutcomeTreatmentP(vtreat.BinomialOutcomeTreatment):
    """bind in non-tuned parmeters for when grid search clones"""
    
    def __init__(self, *, indicator_min_fraction=0.1):
        vtreat.BinomialOutcomeTreatment.__init__(
            self,
            outcome_target=True,
            params = {
                'filter_to_recommended': False,
                'indicator_min_fraction': indicator_min_fraction,
                'coders': {'indicator_code'},
            }
        )

transform = BinomialOutcomeTreatmentP()

clf = Pipeline(steps=[
    ('preprocessor', transform),
    ('classifier', LogisticRegression(solver = 'lbfgs'))]
)

X_train, X_test, y_train, y_test = train_test_split(d_x, d_y, test_size=0.5)

Set up a [cross-validated grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) for hyper parameters, including the indicator coding strategy.

In [7]:
parameters = {
    'preprocessor__indicator_min_fraction': [0.01, 0.1],
    'classifier__C': [0.1, 1],
}

cgm = GridSearchCV(clf, parameters, cv=5)

In [8]:
cgm.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        __main__.BinomialOutcomeTreatmentP(outcome_target=True, )),
                                       ('classifier', LogisticRegression())]),
             param_grid={'classifier__C': [0.1, 1],
                         'preprocessor__indicator_min_fraction': [0.01, 0.1]})

In [9]:
cgm.best_params_

{'classifier__C': 0.1, 'preprocessor__indicator_min_fraction': 0.1}

In [10]:
cgm.cv_results_

{'mean_fit_time': array([0.71360078, 0.28696437, 0.68817482, 0.28684545]),
 'std_fit_time': array([0.03392186, 0.00624858, 0.01178248, 0.01096785]),
 'mean_score_time': array([0.07794299, 0.03840861, 0.07571516, 0.0383904 ]),
 'std_score_time': array([0.00166816, 0.00067049, 0.00099838, 0.00055073]),
 'param_classifier__C': masked_array(data=[0.1, 0.1, 1, 1],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_preprocessor__indicator_min_fraction': masked_array(data=[0.01, 0.1, 0.01, 0.1],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'classifier__C': 0.1,
   'preprocessor__indicator_min_fraction': 0.01},
  {'classifier__C': 0.1, 'preprocessor__indicator_min_fraction': 0.1},
  {'classifier__C': 1, 'preprocessor__indicator_min_fraction': 0.01},
  {'classifier__C': 1, 'preprocessor__indicator_min_fraction': 0.1}],
 'split0_test_score': array([0.7  , 0.735, 0.66 , 0

In [11]:
cgm.best_score_

0.7150000000000001

In [12]:
param_names = list(cgm.cv_results_['params'][0].keys())
perf_cols = {
    p: [cgm.cv_results_['params'][i][p] for 
          i in range(len(cgm.cv_results_['params']))] for p 
            in param_names
    }
perf_cols.update({
        'mean_test_score': cgm.cv_results_['mean_test_score'],
        'rank_test_score': cgm.cv_results_['rank_test_score'],
        'std_test_score': cgm.cv_results_['std_test_score'],
    })
perf_frame = pandas.DataFrame(perf_cols)
perf_frame.set_index(param_names, inplace=True)
perf_frame

mean_test_score  \
classifier__C preprocessor__indicator_min_fraction                    
0.1           0.01                                            0.711   
              0.10                                            0.715   
1.0           0.01                                            0.688   
              0.10                                            0.711   

                                                    rank_test_score  \
classifier__C preprocessor__indicator_min_fraction                    
0.1           0.01                                                2   
              0.10                                                1   
1.0           0.01                                                4   
              0.10                                                3   

                                                    std_test_score  
classifier__C preprocessor__indicator_min_fraction                  
0.1           0.01                                        0.019079  
              0.10                                        0.017889  
1.0           0.01                                        0.025020  
              0.10                                        0.026153

Notice the `preprocessor__indicator_min_fraction` was chosen to be `0.1`. This a good setting, as it allows in the non-informative rare levels.

We can confirm this by evaluating the model by heand.

In [13]:
clf.set_params(preprocessor__indicator_min_fraction=0.01)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.706

In the above we are using the held-out test data to ensure a reliable estimate of the model quality.  The grid search used cross-validation (both in `vtreat` and in the grid search component) to ensure the same.  Let's also look at this model's performance on training data.

In [14]:
clf.score(X_train, y_train)

/Users/johnmount/Documents/work/pyvtreat/pkg/vtreat/vtreat_api.py:273: UserWarning: possibly called transform on same data used to fit
(this causes over-fit, please use fit_transform() instead)
  warnings.warn(


0.786

Notice `vtreat` generates a warning that the same data was used to both design the variable treatment and treat data (leading to nested model bias).  All the use has to do to avoid this bias is call `.fit_transform(X_train, y_train)` on vtreat intead of calling `.fit(X_train, y_train).transform(X_train)`.  The absence of a warning message helps confirm the user has not done this.

We also confirm this `indicator_min_fraction=0.01` setting lets in very many variables, including the non-informative `lev_r` variables.

In [15]:
transform.score_frame_['variable']

0      x_0_lev_c_3
1      x_0_lev_c_1
2      x_0_lev_c_4
3      x_0_lev_c_0
4      x_0_lev_c_2
          ...     
154    x_3_lev_r_6
155    x_3_lev_r_8
156    x_3_lev_r_7
157    x_3_lev_r_2
158    x_3_lev_r_0
Name: variable, Length: 159, dtype: object

Now let's look at `indicator_min_fraction=0.1`.

In [16]:
clf.set_params(preprocessor__indicator_min_fraction=0.1)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

0.721

We see a slighly better out of sample score.

In [17]:
clf.score(X_train, y_train)

/Users/johnmount/Documents/work/pyvtreat/pkg/vtreat/vtreat_api.py:273: UserWarning: possibly called transform on same data used to fit
(this causes over-fit, please use fit_transform() instead)
  warnings.warn(


0.755

The in-sample score is larger for this worse parameter choice. Over-fit by evaluating on training data perfers the excess variables even though they are useless.  This is why the grid search cross-validates in addition to `vtreat`'s use of out of sample procedures.

We can confirm there are fewer variables for this setting.

In [18]:
transform.score_frame_['variable']

0      x_0_lev_c_3
1      x_0_lev_c_1
2      x_0_lev_c_4
3      x_0_lev_c_0
4      x_0_lev_c_2
5      x_4_lev_c_2
6      x_4_lev_c_0
7      x_4_lev_c_1
8      x_4_lev_c_3
9      x_4_lev_c_4
10    x_4_lev__NA_
11     x_8_lev_c_2
12     x_8_lev_c_3
13     x_8_lev_c_4
14     x_8_lev_c_1
15     x_8_lev_c_0
16     x_5_lev_c_3
17     x_5_lev_c_4
18     x_5_lev_c_0
19     x_5_lev_c_1
20     x_5_lev_c_2
21     x_9_lev_c_1
22     x_9_lev_c_0
23     x_9_lev_c_2
24     x_9_lev_c_4
25     x_9_lev_c_3
26    x_9_lev__NA_
27     x_2_lev_c_0
28     x_2_lev_c_1
29     x_2_lev_c_4
30     x_2_lev_c_2
31     x_2_lev_c_3
32    x_2_lev__NA_
33     x_1_lev_c_3
34     x_1_lev_c_0
35     x_1_lev_c_1
36     x_1_lev_c_4
37     x_1_lev_c_2
38    x_1_lev__NA_
39     x_7_lev_c_0
40     x_7_lev_c_1
41     x_7_lev_c_4
42     x_7_lev_c_2
43     x_7_lev_c_3
44    x_7_lev__NA_
45     x_6_lev_c_3
46     x_6_lev_c_1
47     x_6_lev_c_4
48     x_6_lev_c_2
49     x_6_lev_c_0
50     x_3_lev_c_1
51     x_3_lev_c_0
52     x_3_l

An important point is: putting `vtreat` into the hyper-parameters search is expensive. And it may not be reliable as settings of other variable supression steps (such as the `C` and other regularization details) can mask the value of `vtreat`'s variable pruning.  Variables that the model knows to turn off, don't cause problems.

This is also why we don't consider `vtreat`'s pruning a detailed hyper-parameter. As long as it is set to not "blow up too much" we expect modern regularized methods to tolerate a great variation in the setting. 

Or: 
  * The less sensitive we are to a parameter the less it matters if we are exactly at the optimal setting.
  * Parameters are more difficult to optimize if we are less sensitive to them.
  
We have shown a sucessful operation of `vtreat` in hyper-parameter search.  However in the end we do not recommend this workflow as:

  * It is needlessly expensive, as it re-does many nearly identical variable-prep steps.
  * It may not be needed as one may be able to pick "good enough a parameters ahead of time.
  * It may not set better parameters, as downstream regularization may obscure the correct settings.
  
The workflow we recommend is more like the following.

In [19]:
transform = vtreat.BinomialOutcomeTreatment(
            outcome_target=True,
            params = {
                'filter_to_recommended': True,
                'indicator_min_fraction': 0.1,
            }
        )

X_train_treated = transform.fit_transform(X_train, y_train)

X_train_treated.columns

Index(['x_0_lev_c_3', 'x_4_logit_code', 'x_4_lev_c_3', 'x_8_logit_code',
       'x_8_lev_c_2', 'x_8_lev_c_4', 'x_8_lev_c_1', 'x_8_lev_c_0',
       'x_5_logit_code', 'x_5_lev_c_3', 'x_5_lev_c_4', 'x_5_lev_c_0',
       'x_5_lev_c_2', 'x_9_logit_code', 'x_9_lev_c_1', 'x_9_lev_c_4',
       'x_9_lev_c_3', 'x_2_logit_code', 'x_2_lev_c_1', 'x_2_lev_c_2',
       'x_1_logit_code', 'x_1_lev_c_4', 'x_7_logit_code', 'x_7_lev_c_0',
       'x_7_lev_c_2', 'x_7_lev_c_3', 'x_6_logit_code', 'x_3_logit_code',
       'x_3_prevalence_code', 'x_3_lev_c_2', 'x_3_lev_c_3'],
      dtype='object')

In [20]:
clf = Pipeline(steps=[
    ('classifier', LogisticRegression(solver = 'lbfgs'))]
)

parameters = {
    'classifier__C': [0.1, 1],
}

cgm = GridSearchCV(clf, parameters, cv=5)

cgm.fit(X_train_treated, y_train)

cgm.best_params_

{'classifier__C': 0.1}

In [21]:
est = cgm.best_estimator_

X_test_treated = transform.transform(X_test)

est.score(X_test_treated, y_test)

0.713

Note, this didn't achieve the optimal score (likely the `'filter_to_recommended': True` was too strict).  But it is in the ballpark and improves as we get more data.